# boTeach

## Instalación de las librerías requeridas (PyPDF2, langchain, sentence-transformers, chromadb, ollama)

In [2]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install PyPDF2 langchain sentence-transformers chromadb ollama

## Importación de las librerías requeridas (PyPDF2, langChain, sentence-transformers, torch, chromaDB, ollama, re, json)

In [3]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
import torch
import chromadb
import ollama
import re
import json

## IMPLEMENTACIÓN DEL PREPROCESAMIENTO DE DOCUMENTOS

### Función de lectura del PDF

In [4]:
def leer_pdf(ruta_pdf):
    
    # Carga y lectura del PDF desde la ruta especificada
    reader = PdfReader(ruta_pdf) # Abrir el PDF con PyPDF2
    texto = "" # Variable donde se acumulará el contenido extraído del PDF

    # Recorrer todas las páginas del documento
    for pagina in reader.pages: # Recorrer cada página
        pagina_texto = pagina.extract_text()  # Extraer el texto de cada página
        
        if pagina_texto:  #Limpiar el PDF
            # Normalizar bullets y guiones
            pagina_texto = pagina_texto.replace("\uf0b7", "-").replace("‐", "-")
            
            # Quitar números de página
            pagina_texto = re.sub(r'^\s*\d+\s*$', '', pagina_texto, flags=re.MULTILINE)
            
            # Unir líneas que fueron cortadas por saltos de línea dentro del mismo párrafo
            pagina_texto = re.sub(r'(?<!\n)\n(?!\n)', ' ', pagina_texto)
            
            # Mantener saltos dobles para separar párrafos correctamente
            pagina_texto = re.sub(r'\n{2,}', '\n\n', pagina_texto)
            
            # Reemplazar espacios o tabulaciones múltiples por un único espacio
            pagina_texto = re.sub(r'[ \t]+', ' ', pagina_texto)
            
            texto += pagina_texto.strip() + "\n\n"  # Agregar el texto limpio de la página al acumulador, separando por doble salto de línea

    return texto.strip()   # Devolver el texto completo y normalizado del PDF, listo para el proceso de chunking

In [5]:
leer_pdf("EL_CICLO_DEL_AGUA.pdf")

'EL CICLO DEL AGUA Introducción El agua es un recurso esencial para la vida en la Tierra. Sin ella, los seres vivos no podrían desarrollarse ni sobrevivir, y muchos procesos naturales que sostienen los ecosistemas no serían posibles. Aunque el agua parece estar siempre disponible, su cir culación en el planeta es parte de un proceso dinámico y continuo conocido como ciclo del agua o ciclo hidrológico. El ciclo del agua es el proceso mediante el cual el agua se mueve continuamente entre la superficie terrestre, la atmósfera y los océanos, cambiando entre estados líquido, sólido y gaseoso . Comprender el ciclo del agua es fundamental no solo para la biología y la ecología, sino también para actividades humanas como la agricultura, la gestión de recursos hídricos, la planificación urbana y la prevención de desastres naturales. 1. Etapas del ciclo del agua 1.1 Evaporación y transpiración − La evaporación es el proceso mediante el cual el agua líquida de ríos, lagos, mares y superficies húm

### Función de división del texto en fragmentos/chunks (Chunking)

In [6]:
def chunking(texto, tamano=900, solapamiento=200, separator="\n\n"):
    
    # Crear un objeto que divide el texto en fragmentos (chunks)
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=tamano,   # Cantidad máxima de caracteres que tendrá cada chunk
        chunk_overlap=solapamiento, # Cantidad de caracteres que se repiten entre chunks consecutivos (para mantener contexto)
        separators=[separator] # Separador preferido para dividir el texto
    )
    
    chunks = splitter.split_text(texto) # Dividir el texto completo en una lista de chunks
    return chunks # Devolver la lista de chunks 


### Modelo de embeddings: all-MiniLM-L6-v2

In [7]:
modelo_emb = SentenceTransformer('all-MiniLM-L6-v2')  # VARIABLE GLOBAL

### Función de creación de embeddings

In [8]:
def crear_embeddings(chunks):
    embeddings = modelo_emb.encode(chunks, convert_to_tensor=True)  # Generar embeddings para cada chunk usando el modelo previamente cargado
    return embeddings # Devolver los embeddings generados (tensor con un vector por cada chunk)

### Función de almacenamiento de chunks y embeddings asociados a cada chunk en Chroma, la base de datos vectorial

In [9]:
def almacenamiento_en_chroma(chunks, embeddings, nombre_coleccion="boTeach"):
    cliente = chromadb.Client()

    if nombre_coleccion in [col.name for col in cliente.list_collections()]:
        cliente.delete_collection(nombre_coleccion) # Si ya existe una colección con el mismo nombre, se elimina para crearla nuevamente desde cero

    coleccion = cliente.create_collection(name=nombre_coleccion) # Crear una nueva colección donde se guardarán los textos y sus embeddings
    coleccion.add(
        documents=chunks, # Lista de chunks
        embeddings=embeddings.cpu().numpy(), # Convertir los tensores a arrays de NumPy para almacenarlos
        ids=[str(i) for i in range(len(chunks))]  # Asignar un ID único a cada chunk
    )
    return coleccion   # Se devuelve la colección creada, que contiene cada chunk con su embedding asociado

## IMPLEMENTACIÓN DE LA RECUPERACIÓN DE LA INFORMACIÓN

### Función de comparación de similitud de embeddings para obtener los chunks relevantes

In [10]:
def comparar_embeddings_similares(coleccion, texto_consulta, top_k=5):
    
    # Convertir la pregunta del usuario en su embedding, en formato tensor para cálculos eficientes
    embedding_consulta = modelo_emb.encode([texto_consulta], convert_to_tensor=True)
    
    resultados = coleccion.query( # Consultar en la colección de Chroma los embeddings del PDF más similares al embedding de la pregunta
        query_embeddings = embedding_consulta.cpu().numpy(), # Se convierte el tensor a un array de NumPy
        n_results = top_k # Cantidad de chunks del PDF más relevantes que se desean recuperar
    )
    
    return resultados['documents'] # Devolver los chunks del PDF más similares

## IMPLEMENTACIÓN DE LA GENERACIÓN DE RESPUESTAS

### Función de respuesta (Pista y respuesta correcta) con Ollama (gemma:7b-instruct)

In [11]:
def generar_pista_y_respuesta(chunks_relevantes, pregunta):
    
    # Unir todos los chunks relevantes en un solo bloque de texto para pasarlo al modelo
    contexto = "\n\n".join([texto for grupo in chunks_relevantes for texto in grupo])
    ollama_cliente = ollama.Client()
    
    # Prompt diseñado para que el modelo devuelva un JSON limpio con pista y respuesta
    respuesta = ollama_cliente.chat(
        model="gemma:7b-instruct", # Modelo de IA que genera las respuestas de boTeach
        messages=[
            {"role": "system", "content": """
                Eres un asistente educativo que responde preguntas basadas únicamente en un PDF.
                Debes devolver **solo JSON válido**, sin etiquetas ni asteriscos, con esta estructura:
                
                {
                  "pista": "una breve pista para que el estudiante piense la respuesta",
                  "respuesta": "respuesta completa según el contexto, o 'No puedo responder con la información que tengo.'"
                }
                
                REGLAS PARA LA PISTA:
                1. Responde siempre en español. 
                2. Siempre das solo **una pista breve** (1 o 2 frases máximo).
                3. Nunca inventes información ni reveles la respuesta.
                4. Si la pregunta no tiene información disponible, responde exactamente: 
                   "La información no está disponible".
                5. Mantén la pista **neutral y aplicable a cualquier tema**, no menciones secciones o palabras específicas del material.
                
                FORMATO DE RESPUESTA:
                **Pista:** [una sola pista breve]
                
                EJEMPLOS CON PREGUNTAS GENÉRICAS:
                
                Pregunta: ¿Cómo se produce la energía en un organismo?  
                Respuesta: **Pista:** Piensa en los procesos que convierten elementos básicos en energía utilizable.  
                
                Pregunta: ¿Qué factores afectan el crecimiento de las plantas?  
                Respuesta: **Pista:** Considera lo que las plantas necesitan para desarrollarse correctamente.  
                
                Pregunta: ¿Por qué algunas sustancias se mezclan mejor que otras?  
                Respuesta: **Pista:** Reflexiona sobre las propiedades de los materiales y cómo interactúan.  
                
                Pregunta: ¿Qué pasa si falta un recurso natural importante?  
                Respuesta: **Pista:** Imagínate cómo la falta de algo esencial podría alterar un sistema.  
                
                Pregunta: ¿Cuál es la causa de un fenómeno físico?  
                Respuesta: **Pista:** Piensa en los elementos que normalmente generan ese tipo de cambio.

                REGLAS PARA LA RESPUESTA:
                1. Basarse únicamente en el contexto proporcionado.
                2. Si no hay información suficiente, escribe exactamente: "No puedo responder con la información que tengo."
                3. Nunca inventes información.
                """}, 
            {"role": "user", "content": f"Contexto:\n{contexto}\n\nPregunta: {pregunta}"}
        ]
    )
    
    contenido = respuesta["message"]["content"].strip() # Obtener el texto que devuelve el modelo

    # Intentar parsear directamente el JSON generado por el modelo
    try:
        datos = json.loads(contenido) # El JSON se convierte en un diccionario
        pista = datos.get("pista", "La información no está disponible") # Texto de la pista
        respuesta_oficial = datos.get("respuesta", "No puedo responder con la información que tengo.") # Texto de la respuesta correcta
        
    except json.JSONDecodeError:
        # Manejo seguro en caso de que el modelo no devuelva un JSON válido
        pista = "La información no está disponible"
        respuesta_oficial = "No puedo responder con la información que tengo."
    
    return pista, respuesta_oficial #Devolver pista y respuesta correcta como strings

### Función de evaluación de respuesta del usuario

In [12]:
def evaluar_correcta_embeddings(respuesta_usuario, respuesta_oficial, umbral=0.5):
    
    # Convertir las respuestas a embeddings, en formato tensor para cálculos eficientes
    embedding_usuario = modelo_emb.encode(respuesta_usuario, convert_to_tensor=True)
    embedding_oficial = modelo_emb.encode(respuesta_oficial, convert_to_tensor=True)
    
    # Calcular similitud coseno
    similitud = util.cos_sim(embedding_usuario, embedding_oficial).item()
    
    return similitud >= umbral, similitud  # Devolver si la respuesta supera el umbral de similitud (en formato bool) y el valor de similitud

## IMPLEMENTACIÓN DE LA INTERFAZ DE USUARIO

### Función conversar

In [13]:
def conversar(coleccion):
    print("boTeach: ¡Hola! Puedo responder preguntas basándome en tu PDF. Escribí 'salir' para terminar.\n")
    historial = {} # Almacena la pregunta actual y la respuesta oficial del modelo para evaluar al usuario

    entrada = input("Tú: ") 
    while entrada.lower() != "salir": #Si la entrada del usuario indica "salir" se termina la conversación con boTeach           

        if "pregunta" not in historial: # Si aún no hay una pregunta activa, tratamos la entrada como pregunta
            pregunta = entrada
            chunks_relevantes = comparar_embeddings_similares(coleccion, pregunta) # Recuperar chunks relevantes según la pregunta
            # Generar pista y respuesta oficial basada en el contexto recuperado
            pista, respuesta_oficial = generar_pista_y_respuesta(chunks_relevantes, pregunta)

            # Guardar en historial para evaluar la próxima respuesta del usuario
            historial["pregunta"] = pregunta
            historial["respuesta_oficial"] = respuesta_oficial
            
            print(f"boTeach: {pista}\n") # Mostrar al usuario solo la pista

        else: # Si ya hay una pregunta activa, evaluamos la respuesta del usuario
            respuesta_usuario = entrada
            respuesta_oficial = historial["respuesta_oficial"]

            # Evaluar semánticamente si la respuesta del usuario coincide con la oficial
            correcta, similitud = evaluar_correcta_embeddings(respuesta_usuario, respuesta_oficial)
            evaluacion = "CORRECTO" if correcta else "INCORRECTO"
            
            print(f"boTeach: {evaluacion} (similitud: {similitud:.2f}). {respuesta_oficial}\n") # Mostrar resultado y la respuesta oficial

            historial.clear()  # Limpiar historial para procesar la siguiente pregunta

        entrada = input("Tú: ") 

    print("boTeach: ¡Hasta luego!")

In [14]:
# Ruta de tu PDF
ruta_pdf = "EL_CICLO_DEL_AGUA.pdf"

# Leer PDF
texto = leer_pdf(ruta_pdf)

# Dividir en chunks
chunks = chunking(texto)

# Crear embeddings
embeddings = crear_embeddings(chunks)

# Guardar en Chroma
coleccion = almacenamiento_en_chroma(chunks, embeddings)



## EVALUACIÓN DEL CHATBOT

In [14]:
conversar(coleccion)

boTeach: ¡Hola! Puedo responder preguntas basándome en tu PDF. Escribí 'salir' para terminar.



Tú:  ¿Por qué el agua es tan importante para la vida en la Tierra?


boTeach: Considera las funciones esenciales que el agua tiene para los organismos vivos.



Tú:  El agua es importante para la vida en la Tierra porque permite que los seres vivos se hidraten, transporten nutrientes, mantengan sus hábitats y realicen procesos esenciales como la fotosíntesis y la regulación de temperatura.


boTeach: CORRECTO (similitud: 0.81). El agua es esencial para la vida en la Tierra porque es necesario para hidratarse, transportar nutrientes y mantener los hábitats de los organismos vivos.



...

boTeach: La información sobre la capital de Argentina no está incluida en el contexto proporcionado.



Tú:  -


boTeach: INCORRECTO (similitud: 0.10). La información no está disponible.



Tú:  salir


boTeach: ¡Hasta luego!


## INICIA TU CHAT

In [ ]:
conversar(coleccion)